Using unitary gates of 1 qubit to reach a desired target state, the rewards are determined by fidelity

Import Packages

In [595]:
from numpy import *
import cmath
from random import randrange
import pandas as pd
import time
from IPython.display import clear_output
import scipy

Define gates and states

For fidelity measures - introduce pure states other than 0,1,+,- by finding a^2 + b^2 = 1

In [507]:
aar = []
bar = []
for a in range(0,3):
    at = a/3
    aar.append(at)
    b = sqrt(1 - at**2)
    bar.append(b)

In [504]:
one = array([[int(0)], [int(1)]])
zero = array([[int(1)], [int(0)]])
psi1 = (1/(sqrt(2)))*(zero + one)
psi1

array([[0.70710678],
       [0.70710678]])

In [3]:
X = array([[0 , 1], [1, 0]])
Z = array([[1,0],[0,-1]])
Iden = array([[1 , 0], [0, 1]])
Y = array([[0 , -1j], [1j, 0]])
H = (1/(sqrt(2)))*array([[1 , 1], [1, -1]])
S = array([[1 , 0], [0, 1j]])


Create list of actions and define fidelity measure for pure states

In [5]:
list2 = [Iden, X, Y, Z, H, S]

In [468]:
def mag(state):
    prod = (transpose(state.conjugate()).dot(state))*(transpose(state.conjugate()).dot(state))
    return(asscalar(prod))

In [321]:
def fidelityp(guess, right):
    mult = (transpose(guess.conjugate())).dot(right)*(transpose(right.conjugate())).dot(guess)
    #print(mult)
    root = asscalar((scipy.sqrt(mult)))
    return(abs(root))

First set up the dataframe with the correct columns and some initial states

In [514]:
Q_table = pd.DataFrame()

In [515]:
for i in range(0,len(aar)):
    new4 = aar[i]*zero + bar[i]*one
    for m in range(0,len(list2)):
        gate = m
        new = (list2[m].dot(new4))
        Q = pd.DataFrame({'state': str(new4), 'gate':gate, 'new':str(new) ,'Q':0}, index=[0])
        Q_table = Q_table.append(Q)


It's easier to call upon cells in string format rather than array format

In [516]:

Q_tablestr = Q_table.drop_duplicates()
Q_tablestr = Q_tablestr.reset_index(drop=True)
Q_tablestr

,state,gate,new,Q
0,[[0.]\n [1.]],0,[[0.]\n [1.]],0
1,[[0.]\n [1.]],1,[[1.]\n [0.]],0
2,[[0.]\n [1.]],2,[[0.-1.j]\n [0.+0.j]],0
3,[[0.]\n [1.]],3,[[ 0.]\n [-1.]],0
4,[[0.]\n [1.]],4,[[ 0.70710678]\n [-0.70710678]],0
5,[[0.]\n [1.]],5,[[0.+0.j]\n [0.+1.j]],0
6,[[0.33333333]\n [0.94280904]],0,[[0.33333333]\n [0.94280904]],0
7,[[0.33333333]\n [0.94280904]],1,[[0.94280904]\n [0.33333333]],0
8,[[0.33333333]\n [0.94280904]],2,[[0.-0.94280904j]\n [0.+0.33333333j]],0
9,[[0.33333333]\n [0.94280904]],3,[[ 0.33333333]\n [-0.94280904]],0


Set variables - these may need to be played about with more to find optimal values

In [594]:
table = Q_tablestr.copy()

In [598]:
alpha = 0.5 #how much we update Q for every new action
gamma = 0.9 #discount factor 

num_repeats = 200
right = 1/sqrt(2)*(zero - one)
table = Q_tablestr.copy()
right = X.dot(H.dot(Z.dot(aar[2]*zero + bar[2]*one)))

In [599]:

for repeats in range(0,num_repeats):
    k = randrange(len(aar))# choose a random state from list to start 
    #k=2
    psiold = aar[k]*zero + bar[k]*one
    reward = 0 # reset award
    for t in range(0,5): #begin steps
        reward += 0.1 #add more penalty with each step
        
        sec = table[table['state']==str(psiold)] #choose the part of the dataframe that contains the current state
        #print(sec)
        gather = pd.DataFrame() #dataframe to collect all possible new states 
        for n in range(0,(len(sec))): #iterate through the rows that contain the current state
            for z in range(0,len(list2)): # find the action associated with the state in each row
                if(sec.iloc[n]['gate']== z):
                    apply = array(list2[z])
                    zed = z
                    
            psinew = apply.dot(psiold) #create new state by applying action
                
                
            news = table[table['state']==str(psinew)] #choose part of dataframe that contains the new state
            #print(news)
            if len(news) == 0: #if the new state is not in the dataframe, insert it with all the possible actions
                for s in range(0,len(list2)):
                    lad = list2[s].dot(psinew)
                    news = pd.DataFrame({'state' : str(psinew), 'gate':s, 'new':str(lad) ,'Q':0}, index=[0])
                    table = table.append(news)
                    
            maxQ = max(news['Q']) #find the max Q value the new state
            gather = gather.append(news).drop_duplicates() #create a dataframe of all possible new states
            plus = 0 #reset award
            if (fidelityp(psiold, right)) > 0.99999999999:
                plus = 5
            elif (fidelityp(psiold, right)) > 0.8:
                plus = fidelityp(psiold, right)
            else:
                plus = 0
            
            table.loc[(table.state == str(psiold)) & (table.gate == zed), 'Q'] += alpha*((plus - reward) + gamma*(maxQ) - sec.iloc[n]['Q'])
            #update the Q value of the old state based on the rewards given by the new state
        
        #print(gather)
        if plus == 5: #If we have reached the state we want we can stop
            print('goal reached')
            #print(psiold)
            break
        else:
            top = gather.loc[(gather.Q == max(gather['Q']))] #collect the max Q value from all possible new states
            toparray =[] #sort these into an array with their associated action
            actarray =[]
            for o in range(0,len(top)):
                for p in range(0,len(list2)):
                    if (str((array(list2[p])).dot(psiold)) == (top.iloc[o]['state'])):
                        toparray.append((array(list2[p])).dot(psiold))
                        actarray.append(p)
        
            garray =[] #create an array of all possible new states with associated actions
            actg = []
            for a in range(0,len(top)):
                for b in range(0,len(list2)):
                    if (str((array(list2[b])).dot(psiold)) == (gather.iloc[a]['state'])):
                        garray.append((array(list2[b])).dot(psiold))
                        actg.append(b)
        
            q = randrange(100)
            if q > 80: #Create a percentage chance that the next state will be one with a max Q value (expoitation) or a state obtained from a random action (exploration)
                r = randrange(len(toparray))
                psiold = toparray[r]
                action = actarray[r]
            else:
                r = randrange(len(garray))
                psiold = garray[r]
                action = actg[r]
        
        #print(str(t) + 'th state is ' + str(psiold) + ' after action ' + str(action))
    print(repeats)

0
1
2
goal reached
3
goal reached
4
goal reached
5
6
7
8
9
10
11
goal reached
12
13
14
15
16
17
18
19
20
21
goal reached
22
23
24
25
26
27
28
goal reached
29
30
31
32
33
34
35
36
37
goal reached
38
39
40
41
42
43
44
45
46
47
48
49
50
goal reached
51
52
53
54
goal reached
55
56
57
goal reached
58
59
goal reached
60
61
goal reached
62
goal reached
63
64
65
66
67
68
69
goal reached
70
71
72
73
74
75
76
77
78
79
80
81
82
goal reached
83
84
goal reached
85
86
87
goal reached
88
goal reached
89
90
91
92
93
goal reached
94
95
96
97
98
99
100
101
102
103
goal reached
104
105
106
107
108
109
110
111
112
113
goal reached
114
goal reached
115
116
117
118
119
120
121
goal reached
122
123
124
125
126
127
128
129
goal reached
130
131
goal reached
132
133
goal reached
134
135
136
goal reached
137
138
139
140
goal reached
141
142
goal reached
143
144
goal reached
145
goal reached
146
147
148
149
goal reached
150
151
152
153
goal reached
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
1

Show the Q values from target state

In [605]:
#table[table['new'] == str(right)]
table[table['state'] == str(right)]

,state,gate,new,Q
0,[[ 0.9984508 ]\n [-0.05564176]],0,[[ 0.9984508 ]\n [-0.05564176]],40.809683
0,[[ 0.9984508 ]\n [-0.05564176]],1,[[-0.05564176]\n [ 0.9984508 ]],4.736115
0,[[ 0.9984508 ]\n [-0.05564176]],2,[[0.+0.05564176j]\n [0.+0.9984508j ]],4.736115
0,[[ 0.9984508 ]\n [-0.05564176]],3,[[0.9984508 ]\n [0.05564176]],4.736115
0,[[ 0.9984508 ]\n [-0.05564176]],4,[[0.66666667]\n [0.74535599]],36.944176
0,[[ 0.9984508 ]\n [-0.05564176]],5,[[0.9984508+0.j ]\n [0. -0.055641...,4.736115


Show state action pairs with highest Q values

In [601]:
table.sort_values("Q", ascending=False).head(20)

,state,gate,new,Q
0,[[ 0.9984508 ]\n [-0.05564176]],0,[[ 0.9984508 ]\n [-0.05564176]],40.809683
0,[[ 0.9984508 ]\n [-0.05564176]],4,[[0.66666667]\n [0.74535599]],36.944176
16,[[0.66666667]\n [0.74535599]],4,[[ 0.9984508 ]\n [-0.05564176]],36.067523
12,[[0.66666667]\n [0.74535599]],0,[[0.66666667]\n [0.74535599]],32.059178
0,[[1.]\n [0.]],3,[[1.]\n [0.]],6.210541
0,[[1.]\n [0.]],0,[[1.]\n [0.]],6.203456
0,[[0.-0.94280904j]\n [0.+0.33333333j]],0,[[0.-0.94280904j]\n [0.+0.33333333j]],5.533330
0,[[1.]\n [0.]],1,[[0.]\n [1.]],5.290047
1,[[0.]\n [1.]],1,[[1.]\n [0.]],5.175499
0,[[ 0.9984508 ]\n [-0.05564176]],3,[[0.9984508 ]\n [0.05564176]],4.736115


Show what actions are taken when following the max Q value for each state action pair

In [602]:
final = pd.DataFrame() #Dataframe to store state action pairs for top Q values for each agent
tots1 = pd.DataFrame() #Dataframe to store all Q values of all agents
for m in range(0,len(aar)):
    oppa = table.loc[(table.state == str(aar[m]*zero + bar[m]*one))] #select the rows with the initial state
    lol = oppa.sort_values("Q", ascending=False) #sort according to Q value
    top = lol.head(1) #Choose the state action pair with the top Q value
    for n in [0, 1, 2, 3]:
        
        oppa1 = table.loc[(table.state == top['new'].values[n])] #Find the state that results from the top action being applied
        lol = oppa1.sort_values("Q", ascending=False) #find the max Q value of next state
        top1 = lol.head(1)
        
        top = top.append(top1) #add it to the dataframe of steps
        #print(top1)
         
    final = final.append(top) #add this agents top first 4 steps to the other agents top first 4 steps

In [603]:
final

,state,gate,new,Q
1,[[0.]\n [1.]],1,[[1.]\n [0.]],5.175499
0,[[1.]\n [0.]],3,[[1.]\n [0.]],6.210541
0,[[1.]\n [0.]],3,[[1.]\n [0.]],6.210541
0,[[1.]\n [0.]],3,[[1.]\n [0.]],6.210541
0,[[1.]\n [0.]],3,[[1.]\n [0.]],6.210541
8,[[0.33333333]\n [0.94280904]],2,[[0.-0.94280904j]\n [0.+0.33333333j]],4.564081
0,[[0.-0.94280904j]\n [0.+0.33333333j]],0,[[0.-0.94280904j]\n [0.+0.33333333j]],5.533330
0,[[0.-0.94280904j]\n [0.+0.33333333j]],0,[[0.-0.94280904j]\n [0.+0.33333333j]],5.533330
0,[[0.-0.94280904j]\n [0.+0.33333333j]],0,[[0.-0.94280904j]\n [0.+0.33333333j]],5.533330
0,[[0.-0.94280904j]\n [0.+0.33333333j]],0,[[0.-0.94280904j]\n [0.+0.33333333j]],5.533330


See that |1> and [[0.-0.94280904j]\n [0.+0.33333333j]]	are the best states obtainable from the other 2 starting states - we can look at the fidelity between them and the target function to see they are quite close
    

In [606]:
fidelityp(zero,right)

0.9984507974857615

In [613]:
fidelityp((-0.94280904j*zero + 0.333333j*one),right)

0.9598956712854326